# Prepare Starting Geometries
Place the CO2 molecule on the surface in each of the possible adsorption sites.
Following [Jia et al.](https://pubs.rsc.org/en/content/articlehtml/2017/cs/c7cs00026j), we have at least 6 possible configurations.

In [1]:
from pathlib import Path
from ase.build import molecule
from ase.io import read
from ase import Atoms
import numpy as np
import json

Configuration

In [2]:
initial_in_plane: int = 2  # Number of supercells in slab direction in initial surface
final_in_plane: int = 2  # Number of supercells in slab direction in final surface
layers: int = 5  # Number of layers in the supercell direction

Get the CO2 energy

In [3]:
co2_energy = json.loads(Path('co2-molecule/co2.json').read_text())['energy']

## Make the functions
Create functions which create each of the surface geometries

Start with getting a handle on the molecule

In [4]:
example_atoms = read('../1_prepare-surfaces/surfaces/EuOsO3_100_2-cells_5-layers/term=0/relaxed.extxyz')

### Monodentate
A single atom from the surface being close to a single atom from the adsorbate

In [5]:
def make_monodentate_on_metal(atoms: Atoms, distance: float = 2.25, perturb: float = 0.01) -> tuple[Atoms, str]:
    """Place the CO2 moelcule such that an oxygen is near the surface metal

    Args:
        atoms: Surface structure after generating the supercell
        distance: Distance between the Oxygen and the metal
        perturb: How much to perturb the geometry of the CO2 after rotation
            to ensure that symmetry is broken
    Returns:
        - New geometry
        - Atom on bonding site
    """

    # Make then rotate the CO2 such that it is parallel to the z axis
    co2 = molecule('CO2')
    co2.rotate(co2.positions[0, :] - co2.positions[1, :], 'z')
    co2.rattle(perturb)

    # Translate such that it is distance above the metal ion on the highest surface
    #  TODO (wardlt): Prepare for alloys
    metal_ids = [x != "O" for x in atoms.symbols]
    metal_ions = atoms.positions[metal_ids, :]
    top_metal = np.argmax(metal_ions[:, 2])
    new_o_position = np.add(metal_ions[top_metal, :], [0, 0, distance])

    lowest_o = np.argmin(co2.positions[:, 2])
    disp = new_o_position - co2.positions[lowest_o, :]
    co2.translate(disp)

    # Concatentate them
    output = atoms.copy()
    output += co2
    output.cell[2, 2] += 3
    return output, atoms.symbols[metal_ids][top_metal]

In [6]:
def make_monodentate_on_oxygen(atoms: Atoms, distance: float = 2.25, perturb: float = 0.001) -> Atoms:
    """Place the CO2 moelcule such that the carbon is near the surface metal

    Args:
        atoms: Surface structure after generating the supercell
        distance: Distance between the Oxygen and the metal
        perturb: How much to perturb the geometry of the CO2 after rotation
            to ensure that symmetry is broken
    Returns:
        - New geometry
        - Atom on bonding site
    """

    # Make then rotate the CO2 such that it is parallel to the x axis
    co2 = molecule('CO2')
    co2.rotate(co2.positions[0, :] - co2.positions[1, :], 'x')
    co2.rattle(perturb)

    # Translate such that it is distance above the metal ion on the highest surface
    #  TODO (wardlt): Prepare for alloys
    o_ions = atoms.positions[[x == "O" for x in atoms.symbols], :]
    top_o = np.argmax(o_ions[:, 2])
    new_c_position = np.add(o_ions[top_o, :], [0, 0, distance])

    disp = new_c_position - co2.positions[0, :]
    co2.translate(disp)

    # Concatentate them
    output = atoms.copy()
    output += co2
    return output, "O"

## &pi; Coordination
The double bond between C and O being close to a single atom on the surface

In [7]:
def make_pi_conjugate_on_metal(atoms: Atoms, distance: float = 2.25, perturb: float = 0.001) -> Atoms:
    """Place the CO2 moelcule such that a C=O bond is near the surface metal

    Args:
        atoms: Surface structure after generating the supercell
        distance: Distance between the Oxygen and the metal
        perturb: How much to perturb the geometry of the CO2 after rotation
            to ensure that symmetry is broken
    Returns:
        New geometry
    """

    # Make then rotate the CO2 such that it is parallel to the x axis
    co2 = molecule('CO2')
    co2.rotate(co2.positions[0, :] - co2.positions[1, :], 'x')
    co2.rattle(perturb)

    # Translate such that it is distance above the metal ion on the highest surface
    #  TODO (wardlt): Prepare for alloys
    metal_ids = [x != "O" for x in atoms.symbols]
    metal_ions = atoms.positions[metal_ids, :]
    top_metal = np.argmax(metal_ions[:, 2])
    new_position = np.add(metal_ions[top_metal, :], [0, 0, distance])

    disp = new_position - co2.positions[:2, :].mean(axis=0)
    co2.translate(disp)

    # Concatentate them
    output = atoms.copy()
    output += co2
    return output, atoms.symbols[metal_ids][top_metal]

In [8]:
def make_pi_conjugate_on_oxygen(atoms: Atoms, distance: float = 2.25, perturb: float = 0.001) -> Atoms:
    """Place the CO2 moelcule such that a C=O bond is near a surface oxygen

    Args:
        atoms: Surface structure after generating the supercell
        distance: Distance between the Oxygen and the metal
        perturb: How much to perturb the geometry of the CO2 after rotation
            to ensure that symmetry is broken
    Returns:
        New geometry
    """

    # Make then rotate the CO2 such that it is parallel to the x axis
    co2 = molecule('CO2')
    co2.rotate(co2.positions[0, :] - co2.positions[1, :], 'x')
    co2.rattle(perturb)

    # Translate such that it is distance above the metal ion on the highest surface
    #  TODO (wardlt): Prepare for alloys
    o_ions = atoms.positions[[x == "O" for x in atoms.symbols], :]
    top_o = np.argmax(o_ions[:, 2])
    new_c_position = np.add(o_ions[top_o, :], [0, 0, distance])

    disp = new_c_position - co2.positions[:2, :].mean(axis=0)
    co2.translate(disp)

    # Concatentate them
    output = atoms.copy()
    output += co2
    return output, "O"

## Bidentate
There are three possible bidentate positions:
1. Such that each oxygen in CO2 is above a different metal
2. Such that the C is above one metal and an O is above another metal
3. Such at C and O are above the same metal

We are skipping this for now. The first two require concurreance between the length of a CO2 molecule and the intermetal spacing. the last would have the same starting geometry as the &pi; conjugation above the metal site

## Create Surfaces
Iterate over all completed surface relaxations to create starting geometries for each.

In [10]:
surface_dir = Path('../1_prepare-surfaces/surfaces/')
for starting_path in surface_dir.glob(f'*_{initial_in_plane}-cells_{layers}-layers/*/relaxed.extxyz'):
    starting = read(starting_path)
    expected_eng = starting.get_potential_energy()

    # Expand the structure if desired
    if initial_in_plane != final_in_plane:
        repeats = final_in_plane // initial_in_plane
        starting *= [repeats, repeats, 1]
        expected_eng *= repeats * repeats
    
    for name, func in [
        ('mono_metal', make_monodentate_on_metal),
        ('mono_oxygen', make_monodentate_on_oxygen),
        ('pi_metal', make_pi_conjugate_on_metal),
        ('pi_oxygen', make_pi_conjugate_on_metal),
    ]:
        # Make the directory
        write_dir = Path('adsorbate') / starting_path.relative_to(surface_dir).parent / f'width-{final_in_plane}' / name
        write_dir.mkdir(parents=True, exist_ok=True)
        
        # Make structure, save it and metadata
        with_co2, bond_atom = func(starting)
        (write_dir / 'metadata.json').write_text(json.dumps({
            'bond_atom': bond_atom,
            'expected_eng': expected_eng + co2_energy
        }))
        if write_dir.exists():
            continue
        with_co2.write(write_dir / 'unrelaxed.extxyz')